### Imports

In [1]:
from twarc.client2 import Twarc2
from twarc.expansions import ensure_flattened
import configparser
import datetime
import json
import pandas as pd

### Get API tokens

In [2]:
# Create a parser object and disable interpolation
parser = configparser.ConfigParser(interpolation=None)
# Read data from 'config.ini' file
parser.read("config.ini")
# Access sections from the configuration file
parser.sections()
# Get 'bearer_token' from twitter section
bearer_token = parser.get('twitter', 'bearer_token')
# Create a Twarc2 instance with twitter credentials
client = Twarc2(bearer_token = bearer_token)

### Search with `twarc2`

In [14]:
"""
Source code from: https://twarc-project.readthedocs.io/en/latest/api/library/
"""
# Search for any tweets matching the query, Twitter API uses a boolean AND by default
query = "jornalista burra lang:pt -is:retweet"
# Specify the start time in UTC for the time period you want tweets fro
start_time = datetime.datetime(2022, 6, 1, 0, 0, 0, 0, datetime.timezone.utc)
# Specify the end time in UTC for the time period you want tweets from
end_time = datetime.datetime(2022, 7, 1, 0, 0, 0, 0, datetime.timezone.utc)

# 'search_results' is a generator, 'max_results' is max tweets per page, 100 max for full archive search with all expansions
search_results = client.search_all(query=query, start_time=start_time, end_time=end_time, max_results=10)

tweets_list = []
# Get all results page by page:
for page in search_results:
    # Flatten results returning 1 tweet at a time, with expansions inline:
    for tweet in ensure_flattened(page):
        # Append tweets to empty list
        tweets_list.append(tweet)
        print(tweet['text'])
    # Get only one page for testing purposes
    break

@PGMoraisPG @HelemAzevedo "Jornalista";
PARA DE FALAR MAL DO LULA AGORA MESMO VOCÊ É BURRA LIXO CALA A BOCA
@CostaGesuel 👏👏👏👏👏👏👏. Parabéns para jornalista, deu resposta para Anita, que a única coisa que tem naquela mente idiota, burra, é o bumbum, o pior que a juventude hoje não tem mais educação por culpa do PT, não tem mais discernimento para reconhecer o melhor para seu futuro.
@JovemPanNews Essa mulher nem sabe o que está falando kakaka.. jornalista burra da peste!
@JovemPanNews não da pra escutar essa Sra Amanda ou ela é burra ou mau intencionada falar q o Jornalista Alan e foragido é piada ele  é exilado político sua burra
futura jornalista e é burra desse jeito, se preserve https://t.co/SSJYQOHckD
@jogoaberto @hevertonfutebol Imagine ser um jornalista, ser pago pra dar uma opinião e não ter a capacidade cognitiva de entender o que tá respondendo. O cara não entendeu nada que o petraglia disse e quer meter a banca. Mordem a isca muito fácil. Mídia do eixo é sempre muito burra, re

### Process data

In [15]:
# Convert json data to pandas dataframe
# TODO: Process columns with json dictionaries e.g: public_metrics
tweets_df = pd.DataFrame(tweets_list)
# Normalize semi-structured JSON data into a flat table
public_metrics = pd.json_normalize(tweets_df['public_metrics'])
# Concatenate daaframes
tweets_df = pd.concat([tweets_df, public_metrics], axis=1)
# Show dataframe
tweets_df['text']

0    @PGMoraisPG @HelemAzevedo "Jornalista";\nPARA ...
1    @CostaGesuel 👏👏👏👏👏👏👏. Parabéns para jornalista...
2    @JovemPanNews Essa mulher nem sabe o que está ...
3    @JovemPanNews não da pra escutar essa Sra Aman...
4    futura jornalista e é burra desse jeito, se pr...
5    @jogoaberto @hevertonfutebol Imagine ser um jo...
6    Todo dia é uma treta nessa rede social envolve...
7    @FlorbelaEspanc5 @Denise87401709 É lógico e ai...
Name: text, dtype: object